In [4]:
## 问题
    * 我们没有数据
    * 我们不知道哪些特征会影响人找工作，权重如何
    * 人最终面试、上岗受到很多因素影响。如【有其他更优质工作机会、突发事件】等。这些不应该作为【匹配失败】的数据。那么什么样的数据应该作为失败数据呢？上岗成功的一定算是匹配成功，但不匹配的数据仍需要由逻辑得出

## 解题思路
    ### 特征选取
        * 根据现实情况选择部分显著特征，每种特征根据情况折算得分，得分高于 80 的认为最终可以上岗成功（这里把特征总结成枚举项，便于随机生成）
        * 特征之间可能相互影响，如家庭背景影响薪资权重，婚姻状况影响上班耗时权重（暂不考虑）
        * 后续补充权重计算【可能应该折算成薪资后的数额进行】
    ### 数据来源
        * 不依赖外部数据，由现有逻辑生成随机数据源
        * 数据源足够大时(约为 2000 条即可)，可让 AI 掌握由人类总结的逻辑
        * 此时输出结果应与现有逻辑匹配保持一致
    ### 后续业务数据集成
        * 最终上岗的业务数据可以直接与生成的随机数据源混合【暂定30%的数据采用真实业务数据】，训练后查看结果，如果仍获得较高拟合度，则认为我们总结的特征及权重符合实际。如果发现拟合度较低，分析原因
        * 后续持续更换特征，迭代找出更合适的模型
        * 着重分析最终上岗数据中的较低分值的数据，发掘其中隐藏的特征


In [5]:
import numpy as np
import pandas as pd

# 按照以下逻辑生成数据作为基础数据，表达逻辑匹配推荐算法
SIZE = 2000
# 基准逻辑算法
dict_raws_base = {
    # 职位类别 
    # 0 非常不满意
    # 1 不满意
    # 2 满意
    'job_categories': {
        'high': 3,
        'score': [0, 50, 100],
    },
    # 缴金
    # 0 公司缴纳+用户在乎
    # 1 公司缴纳+用户不在乎
    # 2 公司缴纳+用户在乎
    # 3 公司不缴纳+用户不在乎
    'insurance': {
        'high': 4,
        'score': [0, 50, 100, 100],
    },
    # 薪资+福利(包吃住等)
    # 0 薪资低于预期+无福利
    # 1 薪资低于预期+有福利
    # 2 薪资达到预期+无福利
    # 3 薪资达到预期+有福利
    'benefits': {
        'high': 4,
        'score': [0, 60, 80, 100],
    },
    # 上班耗时
    # 0 超过2小时
    # 1 超过1小时
    # 2 40分钟以内
    # 3 20分钟以内
    'go_work_time': {
        'high': 4,
        'score': [0, 20, 80, 100],
    },
}

def get_df(dict_raws):
    df_raws = {}
    for key in dict_raws:
        df_raws[key] = np.random.randint(dict_raws[key]['high'], size=(SIZE))
    columns = dict_raws.keys()
    df=pd.DataFrame(df_raws, columns=columns)

    def get_sum_score(d):
        # 得分 0 总分 < 60 不考虑
        # 得分 1 总分 60 ~ 80 可以考虑
        # 得分 2 总分 > 80 有较强上岗可能性
        score = 0
        count = 0
        for key in columns:
            score = dict_raws[key]['score'][d[key]]
            # 暂时不设置权重，认为各特征权重相等 weight = dict_raws[key]['weight']
            count += (score / len(columns))
        if count < 60:
            score = 0
        if count >= 60 and count < 80:
            score = 1
        if count >= 80:
            score = 2
        return score

    df["score"] = df.apply(get_sum_score,axis =1)
    return df

df = get_df(dict_raws_base)

,job_categories,insurance,benefits,go_work_time,score
0,2,3,0,2,1
1,0,0,3,0,0
2,0,1,2,1,0
3,1,0,3,1,0
4,0,0,2,0,0
...,...,...,...,...,...
1995,1,3,1,1,0
1996,0,3,2,0,0
1997,2,1,2,1,1
1998,1,1,2,3,1


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

wine_data = df[columns].values
wine_target = df["score"].values
print(wine_data)
print(wine_target)


[[2 3 0 2]
 [0 0 3 0]
 [0 1 2 1]
 ...
 [2 1 2 1]
 [1 1 2 3]
 [1 1 2 2]]
[1 0 0 ... 1 1 1]


In [7]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(wine_data,wine_target,test_size=0.3)

r_lf = RandomForestClassifier(random_state=0)
r_lf = r_lf.fit(Xtrain,Ytrain)
r_score = r_lf.score(Xtest,Ytest)

print("Random Forest:{}".format(r_score))

Random Forest:1.0


In [8]:
print(r_lf.predict([[2,3,3,3]]))

[2]


In [10]:
## 模型可以转化成代码供程序使用
import m2cgen as m2c
code = m2c.export_to_java(r_lf)
code

   if ((input[2]) <= (2.5)) {\n                    if ((input[2]) <= (1.5)) {\n                        var98 = [1.0, 0.0, 0.0];\n                    } else {\n                        if ((input[1]) <= (1.5)) {\n                            var98 = [1.0, 0.0, 0.0];\n                        } else {\n                            if ((input[3]) <= (0.5)) {\n                                var98 = [1.0, 0.0, 0.0];\n                            } else {\n                                var98 = [0.0, 1.0, 0.0];\n                            }\n                        }\n                    }\n                } else {\n                    if ((input[1]) <= (1.5)) {\n                        var98 = [1.0, 0.0, 0.0];\n                    } else {\n                        var98 = [0.0, 1.0, 0.0];\n                    }\n                }\n            }\n        } else {\n            if ((input[1]) <= (0.5)) {\n                var98 = [1.0, 0.0, 0.0];\n            } else {\n                if ((input[